# Entry G11 notebook: Create the Marvel Multigraph Database

You'll need to follow the directions in [Entry G11](https://julielinx.github.io/blog/g11_create_multigraphdb_desktop/) up through "Fire up the Database." The code below applies starting at the step "Create Multiple Graphs."

*Warning*, to connect to a specific database using the Neo4j Python 4.X Driver, we have to specify it in `driver.session` **not** by using `:use db_name` as we do in the Neo4j Browser.

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('once')

from neo4j import GraphDatabase

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Set up the initial connection

We'll connect directly to the `system` database since it must be specified in the `driver.session` command.

In [3]:
uri = "bolt://localhost:7687"

driver = GraphDatabase.driver(uri, auth=('neo4j', 'password'))
session = driver.session(database="system")

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Work with the `system` database

In [4]:
pd.DataFrame(session.run('''show databases''').data())

,name,address,role,requestedStatus,currentStatus,error,default,home
0,neo4j,localhost:7687,standalone,online,online,,True,True
1,system,localhost:7687,standalone,online,online,,False,False


In [5]:
session.run('''create database bimodal''')
session.run('''create database unimodal''')
session.run('''create database mixmodal''')

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
pd.DataFrame(session.run('''show databases''').data())

,name,address,role,requestedStatus,currentStatus,error,default,home
0,bimodal,localhost:7687,standalone,online,online,,False,False
1,mixmodal,localhost:7687,standalone,online,online,,False,False
2,neo4j,localhost:7687,standalone,online,online,,True,True
3,system,localhost:7687,standalone,online,online,,False,False
4,unimodal,localhost:7687,standalone,online,online,,False,False


## Work with the `bimodal` database

We'll need to close our session with the `system` database, then we can set up a session to the `bimodal` database.

In [7]:
session.close()
session = driver.session(database="bimodal")

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
session.run('''CALL apoc.schema.assert( {},
{Comic:['name'],Hero:['name']})''')

session.run('''CALL apoc.load.csv('https://raw.githubusercontent.com/tomasonjo/neo4j-marvel/master/data/edges.csv') yield map as row WITH row
MERGE (h:Hero {name:row.hero})
MERGE (c:Comic {name:row.comic})
MERGE (h)-[:APPEARS_IN]->(c)
''')

In [9]:
pd.DataFrame(session.run('''MATCH (n:Hero) RETURN n.name as hero LIMIT 5''').data())

,hero
0,24-HOUR MAN/EMMANUEL
1,3-D MAN/CHARLES CHAN
2,4-D MAN/MERCURIO
3,8-BALL/
4,"ABBOTT, JACK"


## Work with the `mixmodal` database

In [10]:
session.close()
session = driver.session(database="mixmodal")

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
session.run('''CALL apoc.schema.assert( {},
{Comic:['name'],Hero:['name']})''')

session.run('''CALL apoc.load.csv('https://raw.githubusercontent.com/tomasonjo/neo4j-marvel/master/data/edges.csv') yield map as row WITH row
MERGE (h:Hero {name:row.hero})
MERGE (c:Comic {name:row.comic})
MERGE (h)-[:APPEARS_IN]->(c)
''')

session.run('''CALL apoc.periodic.iterate('MATCH (h1:Hero)-->(:Comic)<--(h2:Hero) where id(h1) < id(h2) RETURN h1, h2',
'MERGE (h1)-[r:KNOWS]-(h2) on CREATE SET r.weight = 1 on MATCH SET r.weight = r.weight+1', {batchSize:5000, parallel:false, iterateList:True})''')

In [12]:
pd.DataFrame(session.run('''MATCH (n:Hero)-[:KNOWS]-(h:Hero) RETURN n.name as hero, h.name as friend LIMIT 5''').data())

,hero,friend
0,24-HOUR MAN/EMMANUEL,OLD SKULL
1,24-HOUR MAN/EMMANUEL,M'SHULLA
2,24-HOUR MAN/EMMANUEL,KILLRAVEN/JONATHAN R
3,24-HOUR MAN/EMMANUEL,G'RATH
4,24-HOUR MAN/EMMANUEL,"FROST, CARMILLA"


In [13]:
pd.DataFrame(session.run('''MATCH (n:Hero)-[:APPEARS_IN]-(c:Comic) RETURN n.name as hero, c.name as comic LIMIT 5''').data())

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


## Work with the `unimodal` database

In [14]:
session.close()
session = driver.session(database="unimodal")

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
session.run('''CALL apoc.schema.assert( {},
{Comic:['name'],Hero:['name']})''')

session.run('''CALL apoc.load.csv('https://raw.githubusercontent.com/tomasonjo/neo4j-marvel/master/data/edges.csv') yield map as row WITH row
MERGE (h:Hero {name:row.hero})
MERGE (c:Comic {name:row.comic})
MERGE (h)-[:APPEARS_IN]->(c)
''')

session.run('''CALL apoc.periodic.iterate('MATCH (h1:Hero)-->(:Comic)<--(h2:Hero) where id(h1) < id(h2) RETURN h1, h2',
'MERGE (h1)-[r:KNOWS]-(h2) on CREATE SET r.weight = 1 on MATCH SET r.weight = r.weight+1', {batchSize:5000, parallel:false, iterateList:True})''')

session.run('''MATCH (c:Comic)
DETACH DELETE c''')

In [16]:
pd.DataFrame(session.run('''MATCH (n:Hero)-[:KNOWS]-(h:Hero) RETURN n.name as hero, h.name as friend LIMIT 5''').data())

,hero,friend
0,24-HOUR MAN/EMMANUEL,OLD SKULL
1,24-HOUR MAN/EMMANUEL,M'SHULLA
2,24-HOUR MAN/EMMANUEL,KILLRAVEN/JONATHAN R
3,24-HOUR MAN/EMMANUEL,G'RATH
4,24-HOUR MAN/EMMANUEL,"FROST, CARMILLA"


In [17]:
pd.DataFrame(session.run('''MATCH (n:Hero)-[:APPEARS_IN]-(c:Comic) RETURN n.name as hero, c.name as comic LIMIT 5''').data())

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


""


In [18]:
session.close()

C:\Users\jfish\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
